# Exploratory Data Analysis 

-------------------------------------

by Natalia López Gallego

To create a clean dataset and build a model to predict if a product is **new or used** based on the `condition` column, we will perform an Exploratory Data Analysis first.

In [10]:
import pandas as pd
import numpy as np
from pandas import json_normalize 
import jsonlines
import json

In [ ]:
df = pd.read_json('./MLA_100k.jsonlines', lines=True)

In [3]:
# Detect nested columns
cols_dict = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, dict)).any()]
cols_list = [col for col in df.columns if df[col].apply(lambda x: isinstance(x, list)).any()]

print("Dict type columns (nested):")
print(cols_dict)

print("\nList type columns):")
print(cols_list)


Dict type columns (nested):
['seller_address', 'seller_contact', 'shipping', 'location', 'geolocation']

List type columns):
['sub_status', 'deal_ids', 'non_mercado_pago_payment_methods', 'variations', 'attributes', 'tags', 'coverage_areas', 'descriptions', 'pictures']


In [ ]:
def normalize_json(data):
    """
    Normalizes seller address data and other attributes into a flat format.

    Args:
        data (dict): Dictionary containing nested product data, including seller 
                     address information.

    Returns:
        dict: Dictionary with normalized data, where nested fields have been 
              converted into independent keys.

    Extracted fields:
        - seller_address_comment: Comment on the seller's address.
        - seller_address_longitude: Geographic longitude of the address.
        - seller_address_id: Seller address ID.
        - seller_address_country_name: Seller's country name.
        - seller_address_country_id: Seller's country ID.
        - seller_address_address_line: Seller's address line.
        - seller_address_latitude: Geographic latitude of the address.
        - seller_address_zip_code: Seller's postal code.
        - seller_address_city_name: Seller's city name.
        - seller_address_city_id: Seller's city ID.
        - seller_address_state_name: Seller's state or region name.
        - seller_address_state_id: Seller's state or region ID.
        - seller_address_search_location_neighborhood_name: Neighborhood name based on search location.
        - seller_address_search_location_neighborhood_id: Neighborhood ID based on search location.
        - seller_address_search_location_state_name: State name based on search location.
        - seller_address_search_location_state_id: State ID based on search location.
        - seller_address_search_location_city_name: City name based on search location.
        - seller_address_search_location_city_id: City ID based on search location.
        - warranty: Product warranty information.
        - sub_status: Product sub-status.
        - condition: Product condition (new, used, etc.).
    """

    return {
        "seller_address_comment": data["seller_address"]["comment"],
        "seller_address_longitude": data["seller_address"]["longitude"],
        "seller_address_id": data["seller_address"]["id"],
        "seller_address_country_name": data["seller_address"]["country"]["name"],
        "seller_address_country_id": data["seller_address"]["country"]["id"],
        "seller_address_address_line": data["seller_address"]["address_line"],
        "seller_address_latitude": data["seller_address"]["latitude"],
        "seller_address_zip_code": data["seller_address"]["zip_code"],
        "seller_address_city_name": data["seller_address"]["city"]["name"],
        "seller_address_city_id": data["seller_address"]["city"]["id"],
        "seller_address_state_name": data["seller_address"]["state"]["name"],
        "seller_address_state_id": data["seller_address"]["state"]["id"],
        "seller_address_search_location_neighborhood_name": data["seller_address"]["search_location"]["neighborhood"]["name"],
        "seller_address_search_location_neighborhood_id": data["seller_address"]["search_location"]["neighborhood"]["id"],
        "seller_address_search_location_state_name": data["seller_address"]["search_location"]["state"]["name"],
        "seller_address_search_location_state_id": data["seller_address"]["search_location"]["state"]["id"],
        "seller_address_search_location_city_name": data["seller_address"]["search_location"]["city"]["name"],
        "seller_address_search_location_city_id": data["seller_address"]["search_location"]["city"]["id"],
        "warranty": data["warranty"],
        "sub_status": data["sub_status"],
        "condition": data["condition"]
    }

# Read jsonlines file, normalize each line and save them
with jsonlines.open('./MLA_100k.jsonlines', mode='r') as reader, jsonlines.open('output_normalized.jsonl', mode='w') as writer:
    for obj in reader:
        normalized_data = normalize_json(obj)
        writer.write(normalized_data)


In [ ]:
# Read normalized JSON Lines file
data = []
with jsonlines.open("output_normalized.jsonl", mode="r") as reader:
    for obj in reader:
        data.append(obj)

# Converto to DataFrame
df = pd.DataFrame(data)

In [13]:
df.head()

,seller_address_comment,seller_address_longitude,seller_address_id,seller_address_country_name,seller_address_country_id,seller_address_address_line,seller_address_latitude,seller_address_zip_code,seller_address_city_name,seller_address_city_id,...,seller_address_state_id,seller_address_search_location_neighborhood_name,seller_address_search_location_neighborhood_id,seller_address_search_location_state_name,seller_address_search_location_state_id,seller_address_search_location_city_name,seller_address_search_location_city_id,warranty,sub_status,condition
0,,-58.398671,78162842,Argentina,AR,,-34.62807,,San Cristóbal,TUxBQlNBTjkwNTZa,...,AR-C,San Cristóbal,TUxBQlNBTjkwNTZa,Capital Federal,TUxBUENBUGw3M2E1,Capital Federal,TUxBQ0NBUGZlZG1sYQ,None,[],new
1,,-58.505917,105272448,Argentina,AR,,-34.593552,,Buenos Aires,,...,AR-C,Villa Devoto,TUxBQlZJTDYzNzZa,Capital Federal,TUxBUENBUGw3M2E1,Capital Federal,TUxBQ0NBUGZlZG1sYQ,NUESTRA REPUTACION,[],used
2,,-58.414395,103489016,Argentina,AR,,-34.623391,,Boedo,TUxBQkJPRTQ0OTRa,...,AR-C,Boedo,TUxBQkJPRTQ0OTRa,Capital Federal,TUxBUENBUGw3M2E1,Capital Federal,TUxBQ0NBUGZlZG1sYQ,None,[],used
3,,-58.492921,109827639,Argentina,AR,,-34.628189,,Floresta,TUxBQkZMTzg5MjFa,...,AR-C,Floresta,TUxBQkZMTzg5MjFa,Capital Federal,TUxBUENBUGw3M2E1,Capital Federal,TUxBQ0NBUGZlZG1sYQ,None,[],new
4,,-58.549504,2825909,Argentina,AR,,-34.634655,,Tres de febrero,TUxBQ1RSRTMxODE5NA,...,AR-B,,,Bs.As. G.B.A. Oeste,TUxBUEdSQWVmNTVm,Tres de febrero,TUxBQ1RSRTMxODE5NA,MI REPUTACION.,[],used


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 21 columns):
 #   Column                                            Non-Null Count   Dtype 
---  ------                                            --------------   ----- 
 0   seller_address_comment                            100000 non-null  object
 1   seller_address_longitude                          100000 non-null  object
 2   seller_address_id                                 100000 non-null  object
 3   seller_address_country_name                       100000 non-null  object
 4   seller_address_country_id                         100000 non-null  object
 5   seller_address_address_line                       100000 non-null  object
 6   seller_address_latitude                           100000 non-null  object
 7   seller_address_zip_code                           100000 non-null  object
 8   seller_address_city_name                          100000 non-null  object
 9   seller_address_c

Our DataFrame is comprised by 100000 entries and 21 columns with an `object` data type, and only the `warranty` column has nul values.

In [16]:
df.describe(include='all') 

,seller_address_comment,seller_address_longitude,seller_address_id,seller_address_country_name,seller_address_country_id,seller_address_address_line,seller_address_latitude,seller_address_zip_code,seller_address_city_name,seller_address_city_id,...,seller_address_state_id,seller_address_search_location_neighborhood_name,seller_address_search_location_neighborhood_id,seller_address_search_location_state_name,seller_address_search_location_state_id,seller_address_search_location_city_name,seller_address_search_location_city_id,warranty,sub_status,condition
count,100000,100000.000000,100000,100000,100000,100000,100000.000000,100000,100000,100000,...,100000,100000,100000,100000,100000,100000,100000,39104,100000,100000
unique,282,26316.000000,35922,2,2,1617,26353.000000,319,3656,295,...,25,558,595,29,29,280,313,10264,4,2
top,,-58.381593,151514457,Argentina,AR,,-34.603723,,CABA,,...,AR-C,,,Capital Federal,TUxBUENBUGw3M2E1,Capital Federal,TUxBQ0NBUGZlZG1sYQ,Sí,[],new
freq,99601,6190.000000,933,99997,99997,97782,6198.000000,98730,4098,54574,...,57854,51184,51183,58339,58339,58296,58296,5682,99014,53758


Some columns could be disregarded, but several factors should be considered, such as redundancy, low variability, or limited usefulness for analysis. Here are some observations to help us with this:

1. **Redundancy**: Some columns appear to contain duplicate information. For example:
   - `seller_address_country_name` and `seller_address_country_id` provide the same data in different formats.
   - `seller_address_state_name` and `seller_address_state_id`, as well as city and neighborhood.

2. **Low-information columns**: If a column does not add value to the analysis, it may be unnecessary. For instance:
   - `seller_address_comment`: If these comments are unstructured and not useful for analysis, they might not be needed.
   - `sub_status`: Its relevance depends on the context of your study.

3. **Columns with missing values**: The column `warranty` has many missing values (only 39,104 out of 100,000 records, more than a thrid part). If warranty information is not crucial, it could be disregarded.

## **Data Transformation and Cleaning**

To prepare the data for our model we will follow these steps:

### 1. **Target variable encoding**

* `condition` will be our target variable.
* Encode it as:

  * `"new"` → 1
  * `"used"` → 0


In [ ]:
# Applying the mapping
df['condition'] = df['condition'].map({'new': 1, 'used': 0})

In [27]:
# Verifying the mapping
condition_unique = df['condition'].unique()
print(condition_unique)

[1 0]



### 2. **Drop high-cardinality or redundant columns**

The following columns are either:

* IDs with no predictive power,
* exact duplicates under different names (e.g., multiple city/state fields),
* or long text not relevant for classification.

In [28]:
drop_cols = [
    'seller_address_comment',
    'seller_address_address_line',
    'seller_address_zip_code',
    'seller_address_id',
    'seller_address_city_id',
    'seller_address_state_id',
    'seller_address_country_id',
    'seller_address_search_location_neighborhood_id',
    'seller_address_search_location_city_id',
    'seller_address_search_location_state_id'
]
df = df.drop(columns=drop_cols)

### 3. **Handle missing values**

* `warranty` has many missing values. We could fill with `'no_info'`, but more than a thrid part of the data is missing , so we are going to drop this column.

In [29]:
df = df.drop(columns=['warranty'])

### 4. **Convert coordinates to float**

These may be useful as numeric features:

In [ ]:
df['seller_address_latitude'] = pd.to_numeric(df['seller_address_latitude'], errors='coerce')
df['seller_address_longitude'] = pd.to_numeric(df['seller_address_longitude'], errors='coerce')

### 5. **Encode categorical variables**

We are going to use one-hot encodign for the following variables:

* `seller_address_city_name`
* `seller_address_state_name`
* `seller_address_country_name`
* `seller_address_search_location_*`
* `sub_status`

In [ ]:
categorical_cols = [
    'seller_address_city_name',
    'seller_address_state_name',
    'seller_address_country_name',
    'seller_address_search_location_neighborhood_name',
    'seller_address_search_location_city_name',
    'seller_address_search_location_state_name',
    'sub_status'
]

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)